<a href="https://colab.research.google.com/github/shubh45-nandan/CNN-RL-Hybrid-Models-for-CRC-/blob/main/efficientB4%2BA2C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ==========================================================
# EfficientNetB4 + A2C RL LR-control, full pipeline
# Saves all outputs to Google Drive for GitHub
# ==========================================================

# ---- Optional: installs (uncomment in Colab if needed)
# !pip install -q tensorflow keras opencv-python matplotlib seaborn scikit-learn

import os, shutil, random, zipfile
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB4
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, BatchNormalization, Input
from tensorflow.keras import regularizers, optimizers

print("✅ Imports done")

# ---------------------------
# 0) MOUNT GOOGLE DRIVE (Colab)
# ---------------------------
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=False)
    print("✅ Google Drive mounted at /content/drive")
except Exception as e:
    print("⚠️ Not running inside Colab or google.colab unavailable. Make sure paths are correct.")

# ---------------------------
# 1) PATHS & OUTPUT DIRECTORIES
# ---------------------------
# Your dataset zip path in Drive (change if needed)
ZIP_IN_DRIVE = "/content/drive/MyDrive/archive (1).zip"

if not os.path.exists(ZIP_IN_DRIVE):
    raise FileNotFoundError(f"ZIP not found at: {ZIP_IN_DRIVE}. Update ZIP_IN_DRIVE before running.")

# Where to extract dataset in Colab
EXTRACT_DIR = "/content/Extracted_Kather"

# Where to save ALL outputs for GitHub (in Drive)
SAVE_ROOT = "/content/drive/MyDrive/EffNetB4_A2C_Colorectal"
MODEL_DIR = os.path.join(SAVE_ROOT, "models")
A2C_DIR   = os.path.join(SAVE_ROOT, "a2c_agent")
PLOTS_DIR = os.path.join(SAVE_ROOT, "plots")
HIST_DIR  = os.path.join(SAVE_ROOT, "history")
META_DIR  = os.path.join(SAVE_ROOT, "meta")

for d in [SAVE_ROOT, MODEL_DIR, A2C_DIR, PLOTS_DIR, HIST_DIR, META_DIR]:
    os.makedirs(d, exist_ok=True)

print(f"💾 All outputs will be saved under: {SAVE_ROOT}")

# ---------------------------
# 2) Unzip dataset
# ---------------------------
if os.path.exists(EXTRACT_DIR):
    shutil.rmtree(EXTRACT_DIR)
os.makedirs(EXTRACT_DIR, exist_ok=True)

print(f"📦 Extracting {ZIP_IN_DRIVE} → {EXTRACT_DIR} ...")
with zipfile.ZipFile(ZIP_IN_DRIVE, 'r') as zf:
    zf.extractall(EXTRACT_DIR)
print("✅ Extraction complete")

# Helper: find dataset root (folder with most subfolders)
def find_dataset_root(base):
    best, best_count = base, 0
    for root, dirs, _ in os.walk(base):
        if len(dirs) > best_count:
            best, best_count = root, len(dirs)
    return best

SOURCE_DIR = find_dataset_root(EXTRACT_DIR)
print(f"✅ Dataset root detected at: {SOURCE_DIR}")

# ==========================================================
# PARAMETERS
# ==========================================================
SEED = 42
random.seed(SEED); np.random.seed(SEED); tf.random.set_seed(SEED)

VAL_RATIO = 0.20
TEST_RATIO = 0.15

IMG_SIZE = (380, 380)  # EfficientNetB4 default
BATCH = 32
EXTS = (".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff")

LR_FACTORS = np.array([0.5, 1.0, 2.0], dtype=np.float32)
MIN_LR, MAX_LR = 1e-6, 1e-3

# ==========================================================
# DISCOVER CLASSES & FILES
# ==========================================================
def list_images_recursive(folder):
    out = []
    for root, _, files in os.walk(folder):
        for fn in files:
            if os.path.splitext(fn)[1].lower() in EXTS:
                out.append(os.path.join(root, fn))
    return sorted(out)

class_to_files = {}
for cls in sorted(os.listdir(SOURCE_DIR)):
    p = os.path.join(SOURCE_DIR, cls)
    if os.path.isdir(p):
        files = list_images_recursive(p)
        if files:
            class_to_files[cls] = files

available_classes = sorted(class_to_files.keys())
print("\n📚 Classes detected:")
for c in available_classes:
    print(f"  - {c}: {len(class_to_files[c])} images")

if len(available_classes) == 0:
    raise RuntimeError("❌ No classes found in extracted dataset. Check the zip structure.")

# ==========================================================
# STRATIFIED SPLIT (train / val / test)
# ==========================================================
SPLIT_ROOT = "/content/kather_split_efficientnet_a2c"
TRAIN_DIR, VAL_DIR, TEST_DIR = [f"{SPLIT_ROOT}/{x}" for x in ["train", "val", "test"]]

if os.path.exists(SPLIT_ROOT):
    shutil.rmtree(SPLIT_ROOT)
os.makedirs(TRAIN_DIR); os.makedirs(VAL_DIR); os.makedirs(TEST_DIR)

def allocate_counts(n, val_ratio=VAL_RATIO, test_ratio=TEST_RATIO):
    if n <= 0: return (0,0,0)
    if n == 1: return (1,0,0)
    if n == 2: return (1,0,1)
    test_n = max(1, int(n * test_ratio))
    val_n  = max(1, int(n * val_ratio))
    if test_n + val_n >= n:
        val_n = 1; test_n = 1
    train_n = n - val_n - test_n
    return (train_n, val_n, test_n)

rng = np.random.default_rng(SEED)
totals = {"train":0,"val":0,"test":0}

for cls in available_classes:
    files = class_to_files[cls]
    idx = np.arange(len(files))
    rng.shuffle(idx)

    train_n, val_n, test_n = allocate_counts(len(files))
    train_idx = set(idx[:train_n])
    val_idx   = set(idx[train_n:train_n+val_n])
    test_idx  = set(idx[train_n+val_n:train_n+val_n+test_n])

    os.makedirs(f"{TRAIN_DIR}/{cls}")
    os.makedirs(f"{VAL_DIR}/{cls}")
    os.makedirs(f"{TEST_DIR}/{cls}")

    for i in idx:
        src = files[i]; base = os.path.basename(src)
        if i in train_idx: dst = f"{TRAIN_DIR}/{cls}/{base}"
        elif i in val_idx: dst = f"{VAL_DIR}/{cls}/{base}"
        else:              dst = f"{TEST_DIR}/{cls}/{base}"
        shutil.copy2(src, dst)

    totals["train"] += train_n
    totals["val"]   += val_n
    totals["test"]  += test_n

print("\n✅ Stratified Split Completed:", totals)

# ==========================================================
# DATA GENERATORS
# ==========================================================
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=25, width_shift_range=0.2, height_shift_range=0.2,
    zoom_range=0.3, horizontal_flip=True, vertical_flip=True,
    brightness_range=[0.8,1.2], shear_range=0.2, fill_mode="reflect"
)
val_test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_gen = train_datagen.flow_from_directory(
    TRAIN_DIR, target_size=IMG_SIZE, batch_size=BATCH,
    class_mode='categorical', shuffle=True
)
val_gen   = val_test_datagen.flow_from_directory(
    VAL_DIR, target_size=IMG_SIZE, batch_size=BATCH,
    class_mode='categorical', shuffle=False
)
test_gen  = val_test_datagen.flow_from_directory(
    TEST_DIR, target_size=IMG_SIZE, batch_size=BATCH,
    class_mode='categorical', shuffle=False
)

class_names = list(train_gen.class_indices.keys())
num_classes = len(class_names)
print(f"\n✅ Detected {num_classes} classes: {class_names}")

# ==========================================================
# BUILD EfficientNetB4 MODEL (DenseNet-style head)
# ==========================================================
base_model = EfficientNetB4(weights='imagenet', include_top=False, input_shape=IMG_SIZE + (3,))
base_model.trainable = False

inputs = base_model.input
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)
x = Dense(512, activation='relu', kernel_regularizer=regularizers.l2(1e-4))(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
outputs = Dense(num_classes, activation='softmax')(x)
model = Model(inputs, outputs)

initial_lr = 1e-4
model.compile(optimizer=optimizers.Adam(learning_rate=initial_lr),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

print(model.summary())

# ==========================================================
# A2C AGENT FOR LR ADAPTATION
# ==========================================================
class A2CAgent:
    """
    Advantage Actor-Critic agent that controls LR schedule.
    State: [val_acc, delta_val_acc, normalized_log_lr]
    Actions: multiply LR by LR_FACTORS[action]
    Reward: change in validation accuracy (Δval_acc)
    """
    def __init__(self, state_dim=3, n_actions=3, gamma=0.99,
                 actor_lr=1e-3, critic_lr=1e-3, entropy_coef=0.01):
        self.state_dim = state_dim
        self.n_actions = n_actions
        self.gamma = gamma
        self.entropy_coef = entropy_coef

        # ---- Actor network ----
        s_in = Input(shape=(state_dim,))
        a = Dense(64, activation='relu')(s_in)
        a = Dense(64, activation='relu')(a)
        logits = Dense(n_actions, activation=None)(a)
        self.actor = Model(s_in, logits)

        # ---- Critic network ----
        c_in = Input(shape=(state_dim,))
        c = Dense(64, activation='relu')(c_in)
        c = Dense(64, activation='relu')(c)
        value = Dense(1, activation=None)(c)
        self.critic = Model(c_in, value)

        self.actor_opt = optimizers.Adam(learning_rate=actor_lr)
        self.critic_opt = optimizers.Adam(learning_rate=critic_lr)

    def select_action(self, state):
        """
        state: np.array shape (1, state_dim)
        returns: action index (int), action probabilities (np.array)
        """
        state_tf = tf.convert_to_tensor(state, dtype=tf.float32)
        logits = self.actor(state_tf)
        probs = tf.nn.softmax(logits).numpy()[0]
        probs = np.clip(probs, 1e-8, 1.0)
        probs /= probs.sum()
        action = np.random.choice(self.n_actions, p=probs)  # stochastic policy
        return int(action), probs

    def train(self, state, action, reward, next_state, done):
        """
        One-step A2C update with advantage:
        A = r + γV(s') - V(s)
        Actor loss = -log π(a|s) * A - entropy_bonus
        Critic loss = MSE(target, V(s))
        """
        state_tf = tf.convert_to_tensor(state, dtype=tf.float32)
        next_state_tf = tf.convert_to_tensor(next_state, dtype=tf.float32)
        reward_tf = tf.convert_to_tensor([[reward]], dtype=tf.float32)
        done_tf = tf.convert_to_tensor([[float(done)]], dtype=tf.float32)

        with tf.GradientTape() as tape_a, tf.GradientTape() as tape_c:
            # Critic
            v_s = self.critic(state_tf)          # (1,1)
            v_s_next = self.critic(next_state_tf)
            target = reward_tf + self.gamma * (1.0 - done_tf) * v_s_next
            advantage = target - v_s
            critic_loss = tf.reduce_mean(tf.square(advantage))

            # Actor
            logits = self.actor(state_tf)
            probs = tf.nn.softmax(logits)
            action_one_hot = tf.one_hot([action], self.n_actions)
            selected_prob = tf.reduce_sum(action_one_hot * probs, axis=1, keepdims=True)
            log_prob = tf.math.log(selected_prob + 1e-8)

            entropy = -tf.reduce_sum(probs * tf.math.log(probs + 1e-8), axis=1, keepdims=True)

            actor_loss = -tf.reduce_mean(log_prob * tf.stop_gradient(advantage)) \
                         - self.entropy_coef * tf.reduce_mean(entropy)

        actor_grads = tape_a.gradient(actor_loss, self.actor.trainable_variables)
        critic_grads = tape_c.gradient(critic_loss, self.critic.trainable_variables)
        self.actor_opt.apply_gradients(zip(actor_grads, self.actor.trainable_variables))
        self.critic_opt.apply_gradients(zip(critic_grads, self.critic.trainable_variables))

        return float(actor_loss.numpy()), float(critic_loss.numpy())

def make_state(val_acc, prev_val_acc, lr):
    delta = val_acc - prev_val_acc
    lmin, lmax = np.log10(MIN_LR), np.log10(MAX_LR)
    lrn = (np.log10(lr) - lmin) / (lmax - lmin + 1e-8)
    s = np.array([[val_acc, delta, lrn]], dtype=np.float32)
    return s

def apply_lr_action(cur_lr, action_idx):
    new_lr = float(np.clip(cur_lr * float(LR_FACTORS[action_idx]), MIN_LR, MAX_LR))
    return new_lr

agent = A2CAgent(state_dim=3, n_actions=len(LR_FACTORS),
                 gamma=0.99, actor_lr=1e-3, critic_lr=1e-3)

# ==========================================================
# TRAINING (Phase 1 + Phase 2) – A2C CONTROLS LR
# ==========================================================
current_lr = float(tf.keras.backend.get_value(model.optimizer.learning_rate))
prev_val_acc = 0.0
prev_state = None
prev_action = None

lr_track, val_acc_track, train_acc_track = [], [], []
best_val_acc = 0.0
best_model_path = os.path.join(MODEL_DIR, "best_val_model.h5")

EPOCHS_INITIAL = 15
EPOCHS_FINE = 25

print("\n🚀 Phase 1: Train with EfficientNetB4 base frozen")
for epoch in range(EPOCHS_INITIAL):
    print(f"\n🔁 Epoch {epoch+1}/{EPOCHS_INITIAL} (Phase 1)")
    hist = model.fit(train_gen, validation_data=val_gen, epochs=1, verbose=1)
    train_acc = float(hist.history['accuracy'][-1])
    val_acc = float(hist.history['val_accuracy'][-1])

    current_state = make_state(val_acc, prev_val_acc, current_lr)
    reward = val_acc - prev_val_acc

    if prev_state is not None and prev_action is not None:
        actor_loss, critic_loss = agent.train(prev_state, prev_action, reward,
                                              current_state, done=False)
        print(f"   🧠 A2C update | reward={reward:.5f} | actor_loss={actor_loss:.6f} | critic_loss={critic_loss:.6f}")

    # Save best model by val accuracy
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        model.save(best_model_path)
        print(f"   💾 New best model saved with val_acc={best_val_acc:.4f}")

    action, probs = agent.select_action(current_state)
    new_lr = apply_lr_action(current_lr, action)
    model.optimizer.learning_rate.assign(new_lr)
    current_lr = new_lr

    prev_state = current_state
    prev_action = action
    prev_val_acc = val_acc

    lr_track.append(current_lr)
    val_acc_track.append(val_acc)
    train_acc_track.append(train_acc)

    print(f"   📊 TrainAcc={train_acc:.4f} | ValAcc={val_acc:.4f} | ΔValAcc={reward:.5f} | New LR(A2C)={current_lr:.6f}")

# Terminal update for phase 1
if prev_state is not None and prev_action is not None:
    final_reward = 0.0
    actor_loss, critic_loss = agent.train(prev_state, prev_action, final_reward,
                                          prev_state, done=True)
    print(f"\n✅ Final A2C update Phase 1 | actor_loss={actor_loss:.6f} | critic_loss={critic_loss:.6f}")

print("\n🔓 Phase 2: Fine-tuning EfficientNetB4 top layers")
base_model.trainable = True
for layer in base_model.layers[:-50]:
    layer.trainable = False

model.compile(optimizer=optimizers.Adam(learning_rate=current_lr),
              loss='categorical_crossentropy', metrics=['accuracy'])

for epoch in range(EPOCHS_FINE):
    print(f"\n🧠 Fine Epoch {epoch+1}/{EPOCHS_FINE} (Phase 2)")
    hist = model.fit(train_gen, validation_data=val_gen, epochs=1, verbose=1)
    train_acc = float(hist.history['accuracy'][-1])
    val_acc = float(hist.history['val_accuracy'][-1])

    current_state = make_state(val_acc, prev_val_acc, current_lr)
    reward = val_acc - prev_val_acc

    if prev_state is not None and prev_action is not None:
        actor_loss, critic_loss = agent.train(prev_state, prev_action, reward,
                                              current_state, done=False)
        print(f"   🧠 A2C update | reward={reward:.5f} | actor_loss={actor_loss:.6f} | critic_loss={critic_loss:.6f}")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        model.save(best_model_path)
        print(f"   💾 New best model saved with val_acc={best_val_acc:.4f}")

    action, probs = agent.select_action(current_state)
    new_lr = apply_lr_action(current_lr, action)
    model.optimizer.learning_rate.assign(new_lr)
    current_lr = new_lr

    prev_state = current_state
    prev_action = action
    prev_val_acc = val_acc

    lr_track.append(current_lr)
    val_acc_track.append(val_acc)
    train_acc_track.append(train_acc)

    print(f"   📊 TrainAcc={train_acc:.4f} | ValAcc={val_acc:.4f} | ΔValAcc={reward:.5f} | New LR(A2C)={current_lr:.6f}")

# Final terminal update
if prev_state is not None and prev_action is not None:
    final_reward = 0.0
    actor_loss, critic_loss = agent.train(prev_state, prev_action, final_reward,
                                          prev_state, done=True)
    print(f"\n✅ Final A2C update Phase 2 | actor_loss={actor_loss:.6f} | critic_loss={critic_loss:.6f}")

print(f"\n🏅 Best validation accuracy achieved: {best_val_acc:.4f}")
print(f"🏆 Best model saved at: {best_model_path}")

# ==========================================================
# EVALUATION USING BEST MODEL
# ==========================================================
# Load best model before evaluating on test set
from tensorflow.keras.models import load_model
best_model = load_model(best_model_path)

test_loss, test_acc = best_model.evaluate(test_gen, verbose=1)
print(f"\n✅ Test Accuracy (best model): {test_acc*100:.2f}%")
print(f"✅ Test Loss (best model): {test_loss:.4f}")

y_true = test_gen.classes
y_pred = np.argmax(best_model.predict(test_gen, verbose=0), axis=1)
cm = confusion_matrix(y_true, y_pred)

# ==========================================================
# PLOTS & HISTORY SAVED TO DRIVE
# ==========================================================
# Confusion matrix
cm_path = os.path.join(PLOTS_DIR, "confusion_matrix.png")
plt.figure(figsize=(12,10))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names, yticklabels=class_names,
            annot_kws={"size":10})
plt.title("Confusion Matrix — EfficientNetB4 + A2C", fontsize=16)
plt.xlabel("Predicted", fontsize=14)
plt.ylabel("Actual", fontsize=14)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.savefig(cm_path, dpi=300)
plt.close()
print(f"💾 Saved confusion matrix to: {cm_path}")

# Val accuracy per epoch
acc_path = os.path.join(PLOTS_DIR, "val_accuracy_per_epoch.png")
plt.figure(figsize=(8,5))
plt.plot(val_acc_track, marker='o')
plt.title("Validation Accuracy per Epoch", fontsize=14)
plt.xlabel("Epoch", fontsize=12)
plt.ylabel("Validation Accuracy", fontsize=12)
plt.grid(True, linestyle='--', alpha=0.4)
plt.tight_layout()
plt.savefig(acc_path, dpi=300)
plt.close()
print(f"💾 Saved validation accuracy plot to: {acc_path}")

# Learning rate per epoch
lr_path = os.path.join(PLOTS_DIR, "lr_per_epoch.png")
plt.figure(figsize=(8,5))
plt.plot(lr_track, marker='o')
plt.title("Learning Rate per Epoch (A2C)", fontsize=14)
plt.xlabel("Epoch", fontsize=12)
plt.ylabel("Learning Rate", fontsize=12)
plt.yscale('log')
plt.grid(True, which='both', linestyle='--', alpha=0.4)
plt.tight_layout()
plt.savefig(lr_path, dpi=300)
plt.close()
print(f"💾 Saved learning rate plot to: {lr_path}")

# Combined dashboard
combined_path = os.path.join(PLOTS_DIR, "accuracy_and_lr.png")
plt.figure(figsize=(14,5))
plt.subplot(1,2,1)
plt.plot(val_acc_track, marker='o'); plt.title("Validation Accuracy per Epoch"); plt.grid(True)
plt.subplot(1,2,2)
plt.plot(lr_track, marker='o'); plt.yscale('log'); plt.title("Learning Rate per Epoch (A2C)"); plt.grid(True)
plt.tight_layout()
plt.savefig(combined_path, dpi=300)
plt.close()
print(f"💾 Saved combined plot to: {combined_path}")

# Save training history CSV
hist_df = pd.DataFrame({
    "epoch": np.arange(1, len(val_acc_track)+1),
    "train_accuracy": train_acc_track,
    "val_accuracy": val_acc_track,
    "learning_rate": lr_track
})
hist_csv = os.path.join(HIST_DIR, "training_history.csv")
hist_df.to_csv(hist_csv, index=False)
print(f"💾 Saved training history CSV to: {hist_csv}")

# ==========================================================
# SAVE MODELS: final model + A2C networks
# ==========================================================
final_model_path = os.path.join(MODEL_DIR, "final_model_last_epoch.h5")
model.save(final_model_path)
print(f"💾 Saved final (last epoch) model → {final_model_path}")

actor_path = os.path.join(A2C_DIR, "actor_network.h5")
critic_path = os.path.join(A2C_DIR, "critic_network.h5")
agent.actor.save(actor_path)
agent.critic.save(critic_path)
print(f"💾 Saved A2C actor network → {actor_path}")
print(f"💾 Saved A2C critic network → {critic_path}")

# ==========================================================
# SAVE TEXT SUMMARY (for GitHub README / logs)
# ==========================================================
results_txt = os.path.join(META_DIR, "results_summary.txt")
with open(results_txt, "w") as f:
    f.write("EfficientNetB4 + A2C RL Learning-Rate Control\n")
    f.write("=============================================\n\n")
    f.write(f"Best validation accuracy: {best_val_acc:.4f}\n")
    f.write(f"Test accuracy (best model): {test_acc:.4f}\n")
    f.write(f"Test loss   (best model): {test_loss:.4f}\n\n")
    f.write(f"Best model path: {best_model_path}\n")
    f.write(f"Final model path: {final_model_path}\n")
    f.write(f"Actor network path: {actor_path}\n")
    f.write(f"Critic network path: {critic_path}\n")
    f.write(f"Training history CSV: {hist_csv}\n")
    f.write(f"Plots directory: {PLOTS_DIR}\n")

print(f"📝 Saved results summary to: {results_txt}")

print("\n🎉 All done! Everything you need is in your Drive folder:")
print(f"   {SAVE_ROOT}")
print("You can now download this folder and upload it directly to GitHub.")

✅ Imports done
Mounted at /content/drive
✅ Google Drive mounted at /content/drive
💾 All outputs will be saved under: /content/drive/MyDrive/EffNetB4_A2C_Colorectal
📦 Extracting /content/drive/MyDrive/archive (1).zip → /content/Extracted_Kather ...
✅ Extraction complete
✅ Dataset root detected at: /content/Extracted_Kather/Kather_texture_2016_image_tiles_5000/Kather_texture_2016_image_tiles_5000

📚 Classes detected:
  - 01_TUMOR: 625 images
  - 02_STROMA: 625 images
  - 03_COMPLEX: 625 images
  - 04_LYMPHO: 625 images
  - 05_DEBRIS: 625 images
  - 06_MUCOSA: 625 images
  - 07_ADIPOSE: 625 images
  - 08_EMPTY: 625 images

✅ Stratified Split Completed: {'train': 3256, 'val': 1000, 'test': 744}
Found 3256 images belonging to 8 classes.
Found 1000 images belonging to 8 classes.
Found 744 images belonging to 8 classes.

✅ Detected 8 classes: ['01_TUMOR', '02_STROMA', '03_COMPLEX', '04_LYMPHO', '05_DEBRIS', '06_MUCOSA', '07_ADIPOSE', '08_EMPTY']
71686520/71686520 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/s

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 380, 380,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling           │ (None, 380, 380,  │          0 │ input_layer[0][0] │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization       │ (None, 380, 380,  │          7 │ rescaling[0][0]   │
│ (Normalization)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_1         │ (None, 380, 380,  │          0 │ normalization[0]… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv_pad       │ (None, 381, 381,  │          0 │ rescaling_1[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 190, 190,  │      1,296 │ stem_conv_pad[0]… │
│                     │ 48)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 190, 190,  │        192 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 48)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 190, 190,  │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 48)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_dwconv      │ (None, 190, 190,  │        432 │ stem_activation[… │
│ (DepthwiseConv2D)   │ 48)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_bn          │ (None, 190, 190,  │        192 │ block1a_dwconv[0… │
│ (BatchNormalizatio… │ 48)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_activation  │ (None, 190, 190,  │          0 │ block1a_bn[0][0]  │
│ (Activation)        │ 48)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_squeeze  │ (None, 48)        │          0 │ block1a_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reshape  │ (None, 1, 1, 48)  │          0 │ block1a_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reduce   │ (None, 1, 1, 12)  │        588 │ block1a_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_expand   │ (None, 1, 1, 48)  │        624 │ block1a_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_excite   │ (None, 190, 190,  │          0 │ block1a_activati… │
│ (Multiply)          │ 48)               │            │ block1a_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 190, 190,  │      1,152 │ block1a_se_excit

 Total params: 18,732,391 (71.46 MB)

 Trainable params: 1,054,984 (4.02 MB)

...

   🧠 A2C update | reward=0.00100 | actor_loss=-0.010122 | critic_loss=0.000001
   💾 New best model saved with val_acc=0.8780
   📊 TrainAcc=0.7509 | ValAcc=0.8780 | ΔValAcc=0.00100 | New LR(A2C)=0.000001

🧠 Fine Epoch 25/25 (Phase 2)
102/102 ━━━━━━━━━━━━━━━━━━━━ 141s 1s/step - accuracy: 0.7611 - loss: 0.7607 - val_accuracy: 0.8790 - val_loss: 0.4722


   🧠 A2C update | reward=0.00100 | actor_loss=-0.010171 | critic_loss=0.000000
   💾 New best model saved with val_acc=0.8790
   📊 TrainAcc=0.7660 | ValAcc=0.8790 | ΔValAcc=0.00100 | New LR(A2C)=0.000002

✅ Final A2C update Phase 2 | actor_loss=-0.077461 | critic_loss=0.002611

🏅 Best validation accuracy achieved: 0.8790
🏆 Best model saved at: /content/drive/MyDrive/EffNetB4_A2C_Colorectal/models/best_val_model.h5


24/24 ━━━━━━━━━━━━━━━━━━━━ 32s 587ms/step - accuracy: 0.8230 - loss: 0.5820

✅ Test Accuracy (best model): 86.42%
✅ Test Loss (best model): 0.4487
💾 Saved confusion matrix to: /content/drive/MyDrive/EffNetB4_A2C_Colorectal/plots/confusion_matrix.png
💾 Saved validation accuracy plot to: /content/drive/MyDrive/EffNetB4_A2C_Colorectal/plots/val_accuracy_per_epoch.png
💾 Saved learning rate plot to: /content/drive/MyDrive/EffNetB4_A2C_Colorectal/plots/lr_per_epoch.png


💾 Saved combined plot to: /content/drive/MyDrive/EffNetB4_A2C_Colorectal/plots/accuracy_and_lr.png
💾 Saved training history CSV to: /content/drive/MyDrive/EffNetB4_A2C_Colorectal/history/training_history.csv


💾 Saved final (last epoch) model → /content/drive/MyDrive/EffNetB4_A2C_Colorectal/models/final_model_last_epoch.h5
💾 Saved A2C actor network → /content/drive/MyDrive/EffNetB4_A2C_Colorectal/a2c_agent/actor_network.h5
💾 Saved A2C critic network → /content/drive/MyDrive/EffNetB4_A2C_Colorectal/a2c_agent/critic_network.h5
📝 Saved results summary to: /content/drive/MyDrive/EffNetB4_A2C_Colorectal/meta/results_summary.txt

🎉 All done! Everything you need is in your Drive folder:
   /content/drive/MyDrive/EffNetB4_A2C_Colorectal
You can now download this folder and upload it directly to GitHub.


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# ==========================================================
# GENERATE CLASSIFICATION REPORT FROM SAVED MODEL + TEST SET
# ==========================================================

from tensorflow.keras.models import load_model
from sklearn.metrics import classification_report
import numpy as np
import os

# ---- Paths (update ONLY if your directory names differ) ----
BEST_MODEL_PATH = "/content/drive/MyDrive/EffNetB4_A2C_Colorectal/models/best_val_model.h5"
TEST_DIR = "/content/kather_split_efficientnet_a2c/test"
META_DIR = "/content/drive/MyDrive/EffNetB4_A2C_Colorectal/meta"
IMG_SIZE = (380, 380)
BATCH = 32

# ---- Load test set ----
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_gen = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH,
    class_mode='categorical',
    shuffle=False
)

class_names = list(test_gen.class_indices.keys())

# ---- Load best model ----
print("📌 Loading saved best model...")
model = load_model(BEST_MODEL_PATH)

# ---- Predict ----
print("🔍 Predicting on test set...")
y_true = test_gen.classes
y_pred = np.argmax(model.predict(test_gen, verbose=0), axis=1)

# ---- Classification report ----
report = classification_report(
    y_true,
    y_pred,
    target_names=class_names,
    digits=4
)

print("\n📌 Classification Report:\n")
print(report)

# ---- Save to Drive ----
report_path = os.path.join(META_DIR, "classification_report.txt")
with open(report_path, "w") as f:
    f.write("Classification Report — EfficientNetB4 + A2C\n")
    f.write("============================================\n\n")
    f.write(report)

print(f"💾 Classification report saved to: {report_path}")


NameError: name 'ImageDataGenerator' is not defined